In [1]:
import os
os.chdir('..')

In [2]:
from team_functions import *

In [3]:
today = date.today()
print(f'Today is: {today}')

Today is: 2024-03-23


In [4]:
last_games = pd.read_excel('Predictions/games_odds.xlsx', 'Tonight')
prev_games = pd.read_excel('Predictions/games_odds.xlsx', 'Previous')

In [5]:
last_games

,date,visitor,local,v_line,l_line
0,2024-03-22,BOS,DET,1.083333,8.500000
1,2024-03-22,OKC,TOR,1.060606,10.500000
2,2024-03-22,MEM,SAS,3.000000,1.420168
3,2024-03-22,CLE,MIN,3.350000,1.344828
4,2024-03-22,NOP,MIA,2.300000,1.645161
5,2024-03-22,IND,GSW,2.700000,1.487805
6,2024-03-22,LAC,POR,1.100000,7.500000
7,2024-03-22,PHI,LAL,3.400000,1.335570


In [6]:
options = Options()
options.add_argument('--headless')
driver = Firefox(options=options)

results = [0,0,0,0,1,0,1,1,1]
last_games['Results'] = np.array(results)

In [7]:
url = 'https://www.basketball-reference.com/boxscores/'
soup = getSoup(url, driver=driver)

In [8]:
summaries = soup.find('div', {'class': 'game_summaries'})
games = summaries.findAll('table', {'class': 'teams'})

games_df = []
for idx, game in enumerate(games):
    loser = game.find('tr', {'class': 'loser'}).find('td').find('a').get('href').split('/')[2]
    winner = game.find('tr', {'class': 'winner'}).find('td').find('a').get('href').split('/')[2]
    
    game_row = {'Winner': winner, 'Loser': loser}
    games_df.append(game_row)
    
games_df = pd.DataFrame(games_df)

In [9]:
results = []
for team in last_games['visitor']:
    if team in list(games_df['Winner']):
        results.append(1)
    elif team in list(games_df['Loser']):
        results.append(0)
    else:
        print(f'Error checking results of team: {team}')

last_games['Results'] = np.array(results)

In [10]:
last_games

,date,visitor,local,v_line,l_line,Results
0,2024-03-22,BOS,DET,1.083333,8.500000,1
1,2024-03-22,OKC,TOR,1.060606,10.500000,1
2,2024-03-22,MEM,SAS,3.000000,1.420168,1
3,2024-03-22,CLE,MIN,3.350000,1.344828,0
4,2024-03-22,NOP,MIA,2.300000,1.645161,1
5,2024-03-22,IND,GSW,2.700000,1.487805,1
6,2024-03-22,LAC,POR,1.100000,7.500000,1
7,2024-03-22,PHI,LAL,3.400000,1.335570,0


In [11]:
prev_games.tail()

,date,visitor,local,v_line,l_line,Results
266,2024-03-21,BRK,MIL,4.40,1.227273,0
267,2024-03-21,CHI,HOU,2.42,1.588235,0
268,2024-03-21,UTA,DAL,7.50,1.100000,0
269,2024-03-21,NYK,DEN,4.20,1.243902,0
270,2024-03-21,ATL,PHO,4.30,1.235294,0


In [12]:
prev_games = pd.concat([prev_games, last_games])

In [13]:
url = 'https://sportsbook.draftkings.com/leagues/basketball/nba'

In [14]:
driver.get(url)

html = driver.page_source
soup = bs(html, 'lxml')

driver.close()

In [15]:
def american_to_decimal(american):
    '''
    This functions transform an american system odd (e.g. -120) to a decimal system (e.g 1.8)
    
    INPUTS:
    player: row containing player stats
    
    OUTPUT:
    new_player: copy of the player df input with the updated Odd value in Decimal System
    '''
    # Check if the value is positive or negative (e.g. Odd = +100 or Odd = -120)
    # Transform it to decimal with the given formula
    if american > 0:
        decimal = (american/100) + 1
    else:
        decimal = (100/abs(american)) + 1
    
    return decimal

In [16]:
tables = soup.findAll('tbody', {'class': 'sportsbook-table__body'})
rows = []
for table in tables:
    for row in table.findAll('tr'):
        rows.append(row)
        
games, game, idx = [], {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}, 0
for row in rows:
    team_name = row.find('div', {'class': 'event-cell__name-text'}).text.split(' ')
    if len(team_name[0]) == 3:
        team_id = team_name[0]
    else:
        team_id = team_name[0] + team_name[1][0]
    team_id = team_id.replace('BKN', 'BRK').replace('CHA', 'CHO')
    
    try:
        odd = row.find('span', {'class': 'sportsbook-odds american no-margin default-color'}).text.replace('−', '-')

        if idx == 0:
            game['visitor'] = team_id
            game['v_line'] = american_to_decimal(int(odd))
            idx = 1

        elif idx == 1:
            game['local'] = team_id
            game['l_line'] = american_to_decimal(int(odd))
            idx = 0

            games.append(game)
            game = {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}
    except:
        print(f'Error Scraping {team_id} Odds')
        pass
    
games_df = pd.DataFrame(games)
games_df

Error Scraping BOS Odds
Error Scraping CHI Odds
Error Scraping UTA Odds
Error Scraping HOU Odds
Error Scraping DEN Odds
Error Scraping POR Odds


,date,visitor,local,v_line,l_line
0,2024-03-23,BRK,NYK,3.450000,1.327869
1,2024-03-23,SAC,ORL,2.300000,1.645161
2,2024-03-23,CHO,ATL,4.100000,1.253165
3,2024-03-23,PHO,SAS,1.212766,4.600000
4,2024-03-23,TOR,WAS,2.850000,1.444444


In [18]:
with pd.ExcelWriter('Predictions/games_odds.xlsx') as writer:  
    games_df.to_excel(writer, sheet_name='Tonight', index=0)
    prev_games.to_excel(writer, sheet_name='Previous', index=0)